## Zadanie domowe - Algorytm Canny'ego

Celem zadania domowego jest wykonanie pełnej implementacji algorytmu Canny'ego.

W ramach ćwiczenia w trakcie laboratorium wyznaczono obrazy $g_{NH}$ i $g_{NL}$.
Dla przypomnienia:
Można powiedzieć, że na obrazie $g_{NH}$ są "pewne" krawędzie.
Natomiast na $g_{NL}$ "potencjalne".
Często krawędzie "pewne" nie są ciągłe.
Wykorzystuje się więc krawędzie "potencjalne", aby uzupełnić nieciągłości.
Procedura wygląda następująco:
1. Stwórz stos zawierający wszystkie piksele zaznaczone na obrazie $g_{NH}$.
W tym celu wykorzystaj listę współrzędnych `[row, col]`.
Do pobrania elementu z początku służy metoda `list.pop()`.
Do dodania elementu na koniec listy służy metoda `list.append(new)`.
2. Stwórz obraz, który będzie zawierał informację czy dany piksel został już odwiedzony.
3. Stwórz obraz, który zawierać będzie wynikowe krawędzie.
Jej rozmiar jest równy rozmiarowi obrazu.
4. Wykonaj pętlę, która będzie pobierać elementy z listy, dopóki ta nie będzie pusta.
W tym celu najlepiej sprawdzi się pętla `while`.
    - W każdej iteracji pobierz element ze stosu.
    - Sprawdź, czy dany element został już odwiedzony.
    - Jeśli nie został, to:
        - Oznacz go jako odwiedzony,
        - Oznacz piksel jako krawędź w wyniku,
        - Sprawdź otoczenie piksela w obrazie $g_{NL}$,
        - Dodaj do stosu współrzędne otoczenia, które zawierają krawędź.
        Można to wykonać np. pętlą po stworzonym otoczeniu.
7. Wyświetl obraz oryginalny, obraz $g_{NH}$ oraz obraz wynikowy.
8. Porównaj wynik algorytmu z wynikiem OpenCV.

Pomocnicze obrazy $g_{NH}$ i $g_{NL}$ zostały wprowadzone dla uproszczenia opisu.
Algorytm można zaimplementować w bardziej "zwarty" sposób.

Na podstawie powyższego opisu zaimplementuj pełny algorytm Canny'ego.

In [ ]:
import cv2
from matplotlib import pyplot as plt
import numpy as np

dom = cv2.imread('dom.png', cv2.IMREAD_GRAYSCALE)


def canny_edge_detection(img, t_low, t_high=None, k_size=(5, 5)):
    if t_high is None:
        t_high = 2.5 * t_low

    blur_img = cv2.GaussianBlur(img, k_size, 0)

    s1 = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
    s2 = s1.T
    kernel_s1 = s1 / sum([el if el > 0 else 0 for row in s1 for el in row])
    kernel_s2 = s2 / sum([el if el > 0 else 0 for row in s2 for el in row])
    g_x = np.abs(cv2.filter2D(src=blur_img, ddepth=-1, kernel=kernel_s2))
    g_y = np.abs(cv2.filter2D(src=blur_img, ddepth=-1, kernel=kernel_s1))

    M = np.sqrt(g_x ** 2 + g_y ** 2)
    alfa = np.arctan2(g_y, g_x) / np.pi * 180  # stopnie
    alfa[alfa < 0] += 180

    g_n = np.zeros(M.shape)

    for i in range(1, M.shape[0] - 1):
        for j in range(1, M.shape[1] - 1):
            # frame = M[i - 1: i + 1, i - 1: i + 1]
            # kąt 0
            if 0 <= alfa[i, j] < 22.5 or 157.5 <= alfa[i, j] <= 180:
                if M[i, j - 1] > M[i, j] or M[i, j + 1] > M[i, j]:
                    g_n[i, j] = 0
                else:
                    g_n[i, j] = M[i, j]
            # kąt 45
            elif 22.5 <= alfa[i, j] < 67.5:
                if M[i + 1, j - 1] > M[i, j] or M[i - 1, j + 1] > M[i, j]:
                    g_n[i, j] = 0
                else:
                    g_n[i, j] = M[i, j]
            # kąt 90
            elif 67.5 <= alfa[i, j] < 112.5:
                if M[i + 1, j] > M[i, j] or M[i - 1, j] > M[i, j]:
                    g_n[i, j] = 0
                else:
                    g_n[i, j] = M[i, j]
            # kąt 135
            else:
                if M[i - 1, j - 1] > M[i, j] or M[i + 1, j + 1] > M[i, j]:
                    g_n[i, j] = 0
                else:
                    g_n[i, j] = M[i, j]

    gn_high = (g_n >= t_high)
    gn_low_1 = (g_n < t_high)
    gn_low_2 = (g_n >= t_low)
    gn_low = np.logical_and(gn_low_1, gn_low_2)


    img_wy = np.zeros(img.shape, dtype=np.uint8)
    stack = []
    for i in range(img_wy.shape[0]):
        for j in range(img_wy.shape[1]):
            if gn_high[i, j]:
                stack.append((i, j))

    is_visited = np.zeros(img_wy.shape, dtype=bool)

    while stack:
        x, y = stack.pop()
        if not is_visited[x, y]:
            is_visited[x, y] = True
            img_wy[x, y] = 255
            frame = gn_low[x - 1:x + 1, y - 1:y + 1]
            for i in range(frame.shape[0]):
                for j in range(frame.shape[1]):
                    if frame[i, j]:
                        stack.append((x + i - 1, y + j - 1))

    return img_wy, gn_high

edges, gnh = canny_edge_detection(dom, 4, 11)
plt.imshow(edges, 'gray')
plt.axis('off')
plt.title("Krawędzie")
plt.show()

plt.imshow(gnh, 'gray')
plt.axis('off')
plt.title("gn_h")
plt.show()